<a href="https://colab.research.google.com/github/hemsmalli5/Final-Project---Week1/blob/master/Project_Master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# ***Movie Data Analysis***

Through this project we plan to focus on following business predictions/questions:

Predict popular movie ratings and/or genres within certain release period and intricate genres relationships based on investment and release years.

Analysis specific to one genre, predict if highest budget action movies delivered better revenue?



In [1]:
# Import dependencies
#from google.colab import files
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

# **Import Data**

In [ ]:
# Upload file
#uploaded = files.upload()

In [ ]:
# read the data file
movie_akas = pd.read_csv('Resources/title.akas.tsv', sep='\t')
movie_akas

In [ ]:
# read the data file
movie_basics = pd.read_csv('Resources/title.basics.tsv', sep='\t')
movie_basics

In [ ]:
# read the data file
movie_ratings = pd.read_csv('Resources/title.ratings.tsv', sep='\t')
movie_ratings

In [ ]:
# read the data file
movie_crew = pd.read_csv('Resources/title.crew.tsv', sep='\t')
movie_crew

### Due to the format of the akas file, all rows that do not have US as the region and all movies that have duplicate rows will need to be dropped in SQL before it can be merged with the other datasets. All other ETL steps can take place using Python in the Jupter notebook / CoLab file

### -OR- 

### Save movie_akas_2 as tsv file and load that into SQL

In [ ]:
#Make copy of df for editing
movie_akas_2 = movie_akas.copy()

In [ ]:
# Drop all rows that do not have region as US
movie_akas_2 = (movie_akas_2.loc[movie_akas_2['region'] == 'US'])

# Drop all rows where types is alternative
movie_akas_2 = movie_akas_2[movie_akas_2.types != 'alternative']

# Drop duplicate rows if there is more then one row per movie (keep first row)
movie_akas_2 = movie_akas_2.drop_duplicates(subset=['tconst'], keep='first')
movie_akas_2

# **Merge Datasets**

In [ ]:
# Merge four datasets into one dataframe
movie_data = pd.merge(movie_akas_2, movie_basics, on=["tconst", "tconst"])
movie_data = pd.merge(movie_data, movie_crew, on=["tconst", "tconst"])
movie_data = pd.merge(movie_data, movie_ratings, on=["tconst", "tconst"])

movie_data

# **1. Import SQL Table**

In [ ]:
# For Step 2, will need to establish SQL Database connection and read in dataset from DB
# pd.read_sql_table('table_name', 'postgres:///db_name')

# **2. Clean and Prep Data**

In [ ]:
# Understand Numerical Features
movie_data.info()

In [ ]:
#Make copy of df for editing
movie_data_2 = movie_data.copy()

In [ ]:
# Inspect title types
movie_data_2['titleType'].value_counts()

In [ ]:
# Drop all rows with titleTypes that are not movies from the df
movie_data_2 = (movie_data_2.loc[movie_data_2['titleType'] == 'movie'])

In [ ]:
# Drop all adult film rows from the df
movie_data_2 = (movie_data_2.loc[movie_data_2['isAdult'] == 0])

In [ ]:
# Inspect start years
movie_data_2['startYear'].value_counts()

In [ ]:
# Replace "\N" with date
movie_data_2 = movie_data_2.replace(r'\\N','1700', regex=True)

# Convert columns to int
movie_data_2.startYear = movie_data_2.startYear.astype(int)

# Filter year column
movie_data_2 = (movie_data_2.loc[movie_data_2['startYear'] > 1950])

# Inspect start years
movie_data_2['startYear'].value_counts()

In [ ]:
# Replace 1700 with Nan
movie_data_2 = movie_data_2.replace(r'1700','NaN', regex=True)

In [ ]:
# Drop all rows wih a runtime of NaN
movie_data_2 = (movie_data_2.loc[movie_data_2['runtimeMinutes'] != "NaN"])

# Convert column to int
movie_data_2.runtimeMinutes = movie_data_2.runtimeMinutes.astype(int)

In [ ]:
# Drop all rows wih a director of NaN
movie_data_2 = (movie_data_2.loc[movie_data_2['directors'] != "NaN"])

In [ ]:
# Drop unneeded columns
movie_data_2 = movie_data_2.drop(columns=['title', 'region', 'types', 'tconst', 'titleType', 'originalTitle',
                                          'isAdult','endYear', 'writers','ordering', 'language', 'attributes',
                                          'isOriginalTitle'])
movie_data_2

In [ ]:
# Reset Index
movie_data_2 = movie_data_2.reset_index(drop=True)

In [ ]:
# Rename columns
movie_data_2 = movie_data_2.rename(columns = {"primaryTitle":"Title", "startYear":"Year",
                                              "runtimeMinutes":"Runtime(Min)", "genres":"Genres",
                                              "directors":"Directors", "averageRating":"AverageRating",
                                              "numVotes":"NumVotes"})
movie_data_2

## Split Genres & Director Columns

In [ ]:
# Convert genres column from string into a list
movie_data_2["Genres"] = movie_data_2.Genres.apply(lambda x: x.split(','))
movie_data_2

In [ ]:
# Use MultiLabelBinarizer() to break up and encode genre list 
mlb = MultiLabelBinarizer()
movie_data_2 = movie_data_2.join(pd.DataFrame(mlb.fit_transform(movie_data_2.pop('Genres')), columns=mlb.classes_, index=movie_data_2.index))
movie_data_2

### We can test the ML model with both dropping the Directors column and encoding it to see the difference in accuracy

In [ ]:
# Inspect directors
movie_data_2['Directors'].value_counts()

In [ ]:
# Convert Directors column from string into a list
movie_data_2["Directors"] = movie_data_2.Directors.apply(lambda x: x.split(','))

In [ ]:
# Use MultiLabelBinarizer() to break up and encode Directors list 
mlb = MultiLabelBinarizer()
movie_data_2 = movie_data_2.join(pd.DataFrame(mlb.fit_transform(movie_data_2.pop('Directors')), columns=mlb.classes_, index=movie_data_2.index))
movie_data_2

# **4. Pre-Processing for ML**
## Split & Standardize Data

In [ ]:
# Split preprocessed data into features and target arrays
y = movie_data_2["star_category\r"].values
X = movie_data_2.drop(["star_category\r"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# **5. Create Learning Model**

In [ ]:
# Define the model
number_input_features = len(X_train[0])
nn_model = tf.keras.models.Sequential()

# Add layer to the Sequential model using Keras’ Dense class
nn_model.add(tf.keras.layers.Dense(5, input_dim=number_input_features, activation="relu"))

# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model.summary()

In [ ]:
# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# **6. Train and Test Neural Network**

In [ ]:
# Fit / train the keras model to the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=5)

In [ ]:
# Evaluate model performance using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# **7. Precitions / Conclusion**

# **8. Summary**
